In [ ]:
!pip install osmnx
!pip install networkx
!pip install geopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Calculate shortest path between 2 points using `NetworkX` package

In [ ]:
import pickle
import osmnx as ox
import networkx as nx
ox.config(log_console=True, use_cache=True)
from geopy.geocoders import Nominatim

In [ ]:
# location where you want to find your route
place = 'Berlin, Germany'
# find shortest route based on the mode of travel
mode = 'walk'       # 'drive', 'bike', 'walk'

# create graph from OSM within the boundaries of some geocodable place(s) and save as pickle file
# graph = ox.graph_from_place(place, network_type = mode)
# with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/graph_berlin.obj', 'wb') as fp:
#    pickle.dump(graph, fp)

#load stored graph pickle file
with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/graph_berlin.obj', 'rb') as fp:
  graph_bln = pickle.load(fp)

In [ ]:
# stores the start and end points as geopy.point.Point objects
locator = Nominatim(user_agent = "myapp")

# define the start and end locations as text (user input)
start_location = "Arndtstraße 28, 10965 Berlin"
end_location = "Badensche Straße 52, 10825 Berlin"

# stores the start and end points as geopy.point.Point objects
start_latlng = locator.geocode(start_location).point
end_latlng = locator.geocode(end_location).point

# find the nearest node to the start location
orig_node = ox.get_nearest_node(graph_bln, start_latlng)
# find the nearest node to the end location
dest_node = ox.get_nearest_node(graph_bln, end_latlng)

In [ ]:
# find shortest path based on distance or time
optimizer = 'length'        # 'length','time'

#  find the shortest path
shortest_route = nx.shortest_path(graph_bln,
                                  orig_node,
                                  dest_node,
                                  weight=optimizer)

In [ ]:
import folium

shortest_route_map = ox.plot_route_folium(graph_bln, shortest_route)

# Marker class only accepts coordinates in tuple form
start_latlng = (start_latlng[0],start_latlng[1])
end_latlng   = (end_latlng[0],end_latlng[1])
start_marker = folium.Marker(
            location = start_latlng,
            popup = start_location,
            icon = folium.Icon(color='green'))
end_marker = folium.Marker(
            location = end_latlng,
            popup = end_location,
            icon = folium.Icon(color='red'))
# add the circle marker to the map
start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)
shortest_route_map

### Backup

In [ ]:
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim

ox.config(log_console=True, use_cache=True)

locator = Nominatim(user_agent = "myapp")

# define the start and end locations in latlng
#start_latlng = (37.78497,-122.43327)
#end_latlng = (37.78071,-122.41445)

start_location = "Arndtstraße 23, 10965 Berlin"
end_location = "Arndtstraße 10, 10965 Berlin"

# stores the start and end points as geopy.point.Point objects
start_latlng = locator.geocode(start_location).point
end_latlng = locator.geocode(end_location).point

# location where you want to find your route
place = 'Berlin, Germany'
# find shortest route based on the mode of travel
mode = 'walk'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'

# create graph from OSM within the boundaries of some 
# geocodable place(s)
graph = ox.graph_from_place(place, network_type=mode)

# find the nearest node to the start location
orig_node = ox.distance.nearest_nodes(graph, start_latlng[0], start_latlng[1])

# find the nearest node to the end location
dest_node = ox.distance.nearest_nodes(graph, end_latlng[0], end_latlng[1])

#  find the shortest path
shortest_route = nx.shortest_path(graph,
                                  orig_node,
                                  dest_node,
                                  weight=optimizer)

In [ ]:
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim

ox.config(log_console=True, use_cache=True)

locator = Nominatim(user_agent = "myapp")

# define the start and end locations in latlng
#start_latlng = (37.78497,-122.43327)
#end_latlng = (37.78071,-122.41445)

start_location = "Arndtstraße 23, 10965 Berlin"
end_location = "Arndtstraße 10, 10965 Berlin"

# stores the start and end points as geopy.point.Point objects
start_latlng = locator.geocode(start_location).point
end_latlng = locator.geocode(end_location).point

# location where you want to find your route
place = 'Berlin, Germany'
# find shortest route based on the mode of travel
mode = 'walk'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'

# create graph from OSM within the boundaries of some 
# geocodable place(s)
graph = ox.graph_from_place(place, network_type=mode)

# find the nearest node to the start location
orig_node = ox.distance.nearest_nodes(graph, start_latlng[0], start_latlng[1])

# find the nearest node to the end location
dest_node = ox.distance.nearest_nodes(graph, end_latlng[0], end_latlng[1])

#  find the shortest path
shortest_route = nx.shortest_path(graph,
                                  orig_node,
                                  dest_node,
                                  weight=optimizer)

/usr/local/lib/python3.7/dist-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
